In [19]:
# DATA Segment
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from scaler import StandardScalerLSTM
from model_lstm import PricePredictor, Traning
from run_lstm import get_minmax

SEQ_LEN = 8
BATCH_SIZE = 16
FEATURE_SIZE = 12

In [20]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device = torch.device("cpu")
print(device)

cpu


In [21]:
# device = 'cpu'
path = 'files/lstm_adagrad.pth'
minmax_scaler = get_minmax(device)
model = PricePredictor(minmax_scaler,BATCH_SIZE,device=device).to(device)
model.load_state_dict(torch.load(path))
model.eval()

PricePredictor(
  (lstm): LSTM(9, 150, batch_first=True)
  (linear): Linear(in_features=150, out_features=1, bias=True)
  (ReLU): ReLU()
)

In [22]:
from run_lstm import make_loaders
train_loader,test_loader = make_loaders(batch_size=1)

In [23]:
# loss_function = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
# epochs = 80
# # model = PricePredictor(minmax_scaler,BATCH_SIZE,device=device).to(device)
# trainer = Traning(model, loss_function, optimizer, train_loader, test_loader, epochs=epochs,device=device)
# trainer.train()
# trainer.test()
# trainer.save("kanad")

In [24]:
# torch.save(model.state_dict(), f"files/kanad_1.pth")

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque

class MyModel(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 8)
        self.fc4 = nn.Linear(8, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size
        self.action_size = 3  # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model = MyModel(self.state_size, self.action_size).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.criterion = nn.MSELoss()

        # if is_eval:
        #     self.model.load_state_dict(torch.load(model_name))
        #     self.model.eval()
        # else:
        #     self.model = model

    def act(self, state):
        if not self.is_eval and random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            options = self.model(state)
        return torch.argmax(options, dim=1).item()

    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])

        for state, action, reward, next_state, done in mini_batch:
            state = torch.tensor(state, dtype=torch.float32)
            next_state = torch.tensor(next_state, dtype=torch.float32)
            action = torch.tensor([action], dtype=torch.int64)
            reward = torch.tensor([reward], dtype=torch.float32)

            if not done:
                target = reward.to(device) + self.gamma * torch.max(self.model(next_state).detach()).to(device)
            else:
                target = reward.to(device)

            # Get the output for all actions
            target_f = self.model(state)
            target_f[0][0][action] = target

            # Zero the parameter gradients
            self.optimizer.zero_grad()
            # Forward + backward + optimize
            loss = self.criterion(target_f, self.model(state))
            loss.backward()
            self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [26]:
def formatPrice(n):
    return("-Rs." if n<0 else "Rs.")+"{0:.2f}".format(abs(n))

In [29]:



window_size = 150
episode_count = 100
agent = Agent(window_size)
# data = getStockDataVec(stock_name)
# data = inputs
batch_size = 32
for e in range(episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    # state = getState(data, 0, window_size + 1)
    state = None
    total_profit = 0
    agent.inventory = []
    
    t = len(train_loader)
    count = 0
    for seq, targets in train_loader:
        count = count + 1
        seq, targets = seq.to(device), targets.to(device)
        if state is None:
            action = 1
        else:
            action = agent.act(state)
        pred_price, next_state = model.hidden_predict(seq)
        reward = 0
        if action == 1: # buy
            agent.inventory.append(targets)
            print("Buy: " + formatPrice(targets.item()))
        elif action == 2 and len(agent.inventory) > 0: # sell
            bought_price = window_size_price = agent.inventory.pop(0)
            reward = max(targets - bought_price, 0)
            total_profit += targets - bought_price
            print("Sell: " + formatPrice(targets.item()) + " | Profit: " + formatPrice(targets.item() - bought_price.item()))
        done = True if count == t else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print("--------------------------------")
            print("Total Profit: " + formatPrice(total_profit.item()))
            print("--------------------------------")
        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)
    if e % 10 == 0:
        agent.model.save(str(e))



Episode 0/100
Buy: Rs.1520.77
Buy: Rs.1492.25
Sell: Rs.1502.51 | Profit: -Rs.18.26
Sell: Rs.1489.26 | Profit: -Rs.2.99
Buy: Rs.1451.34
Buy: Rs.1449.05
Buy: Rs.1448.72
Buy: Rs.1439.03
Sell: Rs.1436.51 | Profit: -Rs.14.83
Buy: Rs.1436.23
Sell: Rs.1426.46 | Profit: -Rs.22.59
Sell: Rs.1434.32 | Profit: -Rs.14.40
Buy: Rs.1408.99
Buy: Rs.1402.03
Sell: Rs.1364.59 | Profit: -Rs.74.44
Sell: Rs.1329.78 | Profit: -Rs.106.45
Sell: Rs.1374.17 | Profit: -Rs.34.82
Sell: Rs.1374.62 | Profit: -Rs.27.41
Buy: Rs.1349.97
Sell: Rs.1388.76 | Profit: Rs.38.79
Buy: Rs.1396.93
Buy: Rs.1395.78


/tmp/ipykernel_30081/169715604.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32)
/tmp/ipykernel_30081/169715604.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_state = torch.tensor(next_state, dtype=torch.float32)
/tmp/ipykernel_30081/169715604.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)


RuntimeError: a Tensor with 3 elements cannot be converted to Scalar

In [ ]:



# window_size = 150
# episode_count = 100
# agent = Agent(window_size)
# # data = getStockDataVec(stock_name)
# data = inputs
# batch_size = 32
# for e in range(episode_count + 1):
#     print("Episode " + str(e) + "/" + str(episode_count))
#     # state = getState(data, 0, window_size + 1)
#     state = None
#     total_profit = 0
#     agent.inventory = []
    
#     t = len(train_loader)
#     for seq, targets in train_loader:
#         seq, targets = seq.to(device), targets.to(device)
#         if state is None:
#             action = 1
#         else:
#             action = agent.act(state)
#         pred_price, next_state = model.hidden_predict(seq)
#         reward = 0
#         if action == 1: # buy
#             agent.inventory.append(targets)
#             print("Buy: " + formatPrice(targets.item()))
#         elif action == 2 and len(agent.inventory) > 0: # sell
#             bought_price = window_size_price = agent.inventory.pop(0)
#             reward = max(targets - bought_price, 0)
#             total_profit += targets - bought_price
#             print("Sell: " + formatPrice(targets.item()) + " | Profit: " + formatPrice(targets.item() - bought_price.item()))
#         done = True if count == t else False
#         agent.memory.append((state, action, reward, next_state, done))
#         state = next_state
#         if done:
#             print("--------------------------------")
#             print("Total Profit: " + formatPrice(total_profit.item()))
#             print("--------------------------------")
#         if len(agent.memory) > batch_size:
#             agent.expReplay(batch_size)
#     if e % 10 == 0:
#         agent.model.save(str(e))

